# Zero & Few Shot Prompts : Hugging Face

Demonstrates the zero/few shot prompting technique using the Cohere commands models

**Note**
* Results will vary depending on the model
* To use this notebook you must have the Cohere API key.

#### Google Colab
If you are running the code in Google colab, install the packages by uncommenting/running the cell below

* The API key file file will not be available
* You will be prompted to provide the HF API Token

Uncomment & run the code in the cell below:

In [1]:
## The script is downloaded and run to setup the utils folder

# !curl -H "Accept: application/vnd.github.VERSION.raw" https://raw.githubusercontent.com/acloudfan/gen-ai-app-dev/main/Setup/gcsetup.sh  > gcsetup.sh
# !chmod u+x gcsetup.sh
# !./gcsetup.sh 


## Change the location of the environment file before proceeding

In [2]:
from dotenv import load_dotenv
import os
import sys

import warnings

warnings.filterwarnings("ignore")

# Load the file that contains the API keys
load_dotenv('C:\\Users\\raj\\.jupyter\\.env')


True

In [3]:
# Setting path so we can access the utils folder
sys.path.append('../')
sys.path.append('./')

from utils.api_key_check_utility import api_key_check

# Check if the HF key is available
api_key = api_key_check("HUGGINGFACEHUB_API_TOKEN")

Key:  HUGGINGFACEHUB_API_TOKEN  already set in environment.


In [4]:
from huggingface_hub import InferenceClient

model_id = 'tiiuae/falcon-7b-instruct'

# model_id = 'google/flan-t5-xxl'

client = InferenceClient(model=model_id)

## Try Zero-Shot

In [5]:
prompt = """
Return a list of named entities and their types in the text.

Text: john travelled with his dog duffy from texas to Turkey and met with his friend abdul
"""

client.text_generation(prompt, max_new_tokens=75)

'<pre><code>[John] [Person] [Animal] [Adul] [Friend]\n</code></pre>'

## Instruct the output to be in JSON format

In [6]:
prompt = """
Return a list of named entities and their types in the text.

Format response as JSON

Text: john travelled with his dog duffy from texas to Turkey and met with his friend abdul
Answer:
"""

client.text_generation(prompt, max_new_tokens=100)

'{\n  "named_entities": [\n    {"type": "Person"},\n    {"type": "Location"},\n    {"type": "Animal"},\n    {"type": "Friend"}\n  ],\n  "sentences": [\n    {"type": "Voyage"},\n    {"type": "Meeting"},\n    {"type": "Dog"},\n    {"type": "Turkey"},\n    {"type": "Abdul"}\n  ]\n}'

## 1-Shot

We want the JSON to be in this format: [{entity: type}, {entity : type}....]

All locattions should have a type = place

In [7]:
prompt = """
Return a list of named entities and their types in the text.

Format response as JSON shown in answer

Text: Anil lives in Paris, the most beautiful city in France
Answer: [{'entity': 'Anil', 'type': 'person'}, {'entity': 'Paris', 'type' : 'place'}, {'entity': 'France', 'type' : 'place'}]


Text: john travelled with his dog duffy from texas to Turkey and met with his friend abdul
Answer: 
"""
# Use the stop_sequences parameter to stop the generation after a "]" is output by model
# THERE IS ERROR IN HF DOCUMENTATION - it shows the parameter name as stop instead of stop_sequences
client.text_generation(prompt, max_new_tokens=75,stop_sequences=["]"])

"[{'entity': 'john', 'type': 'person'}, {'entity': 'duffy', 'type': 'animal'}, {'entity': 'dul', 'type': 'person'}, {'entity': 'abdul', 'type': 'person'}]"

## 2-Shot

In [8]:
prompt = """
Return a list of named entities and their types in the text.

Format response as JSON. Stop generating once valid JSON is created.

Text: Anil lives in Paris the most beautiful city in France
Answer: [{'entity': 'Anil', 'type': 'person'}, {'entity': 'Paris', 'type' : 'place'}, {'entity': 'France', 'type' : 'place'}]

Text: Jim lives in texas and he owns a cat named Alexa
Answer: [{'entity': 'Jim', 'type':'person'}, {'entity': 'texas', 'type': 'place'}, {'entity': 'alexa', 'type': 'cat'}]

Text: john travelled with his dog duffy from Texas to Turkey and met with his friend abdul
Answer:
"""

client.text_generation(prompt, max_new_tokens=75, stop_sequences=["]"])

"[{'entity': 'john', 'type': 'person'}, {'entity': 'duffy', 'type': 'animal'}, {'entity': 'turkey', 'type': 'place'}, {'entity': 'abdul', 'type': 'person'}]"

## Few -Shots

In [9]:
prompt = """
Return a list of named entities and their types in the text.

Return a list of named entities and their types in the text.

Format response as JSON. Stop generating once valid JSON is created.

Text: Anil lives in Paris the most beautiful city in France
Answer: [{'entity': 'Anil', 'type': 'person'}, {'entity': 'Paris', 'type' : 'place'}, {'entity': 'France', 'type' : 'place'}]

Text: Jim lives in texas and he owns a cat named Alexa
Answer: [{'entity': 'Jim', 'type':'person'}, {'entity': 'texas', 'type': 'place'}, {'entity': 'alexa', 'type': 'cat'}]

Text: Mary loves her hometown in Calfiornia, where she lives with her husband Paul
Answer: [{'entity': 'Mary', 'type':'person'}, {'entity': 'Calfiornia', 'type': 'state'},{'entity': 'Paul', 'type':'person'}]

Text: Jim is friends with Brad  who lives in LA in California
Answer: [{'entity': 'Jim', 'type':'person'}, {'entity': 'LA', 'type': 'city'}, {'entity': 'Calfiornia', 'type': 'state'}{'entity': 'Brad', 'type':'person'}]


Text: john travelled with his dog duffy from New Jersey to Turkey and met with his friend abdul
Answer:
"""

client.text_generation(prompt, max_new_tokens=75, stop_sequences=["]"])

"[{'entity': 'john', 'type':'person'}, {'entity': 'duffy', 'type': 'animal'}, {'entity': 'Abdul', 'type': 'person'}]"

In [12]:
text = "using the LLM you can build intelligent apps"


prompt = f"what category out of these ['news', 'health', 'finance', 'technology'], does this article belong to : " + text

client.text_generation(prompt)

"\nThe article belongs to the 'finance' category."

## Limitation with few-shot technique

### Few shot may not produce the desired result for reasoning/mathematics/logical problems

Try out this prompt with 

* Chat GPT 3.5

* Chere Command (use playground)

* Google Bard   (uses Gemini AI / LaMDA model)

You will realize that it works for Bard and it doesn't work for Chat GPT 3.5 or Cohere Command!!!

Do your research to figure out why is that the case. Share your results on the Q&A forum.